In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from sandbox import rrtiris

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform, RollPitchYaw
import meshcat
from pydrake.all import GenerateSeedingPolytope
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.multibody.rational_forward_kinematics import FindEpsilonLower, FindEpsilonLowerVector, FindEpsilonUpperVector
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import FindMaxEpsForAllIneqs, MosekSolver, MosekSolverDetails


# Build Plant


In [4]:
q0 = [0.0, 0.0, 0.0]
q_low  = [-1.7, -2., -1.7]
q_high = [ 1.7,  2.,  1.7]


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/oneDOF_iiwa7_with_box_collision.sdf")
twoDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/twoDOF_iiwa7_with_box_collision.sdf")

In [6]:
box_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(twoDOF_iiwa_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset))



locs = [[0.,0.,0.],[0.,.5,0.],[0.,-.5,0.]]
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", models[0]), RigidTransform(locs[0]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_twoDOF_link_0", models[1]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[1]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[2]))


plant.Finalize()

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
            
#q_low = plant.GetPositionLowerLimits().tolist()
#q_high = plant.GetPositionUpperLimits().tolist()

Ratfk = RationalForwardKinematics(plant)


In [ ]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram
visualizer.visualize_collision_constraint(N = 30)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6002...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
Connected to meshcat-server.


In [ ]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

In [ ]:
#filter fused joints self collisions 
digaram = visualizer.diagram
context = visualizer.diagram_context
plant_context = visualizer.plant_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
iiwa_oneDOF_gids = [gid for gid in gids if "iiwa7_oneDOF::" in get_name_of_gid(gid)]
iiwa_twoDOF_gids = [gid for gid in gids if "iiwa7_twoDOF::" in get_name_of_gid(gid)]

oneDOF_fused_col_geom = iiwa_oneDOF_gids[2:]
iiwa_oneDOF_fused_set = GeometrySet(oneDOF_fused_col_geom)
twoDOF_fused_col_geom = iiwa_twoDOF_gids[4:]
iiwa_twoDOF_fused_set = GeometrySet(twoDOF_fused_col_geom)
# print([get_name_of_gid(gid) for gid in oneDOF_fused_col_geom])
# print([get_name_of_gid(gid) for gid in twoDOF_fused_col_geom])
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_oneDOF_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_twoDOF_fused_set))
pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

# Setup IRIS Options and Generate Regions

In [ ]:
# seed_points_q = np.array([[0.0, 0, 0], # zero config
#                         [0.0, -1.3, -1.3],  # start
#                         [0.9, -1.5, -0.8]     # goal
#                          ])
seed_points_q = np.array([[0.0, 0, 0], # zero config
                        [0.8, 1.3, -0.8],  # START: blue low green up
                        [0.1, 0.9, -1.2],     # GOAL: green low other up
                        [0.2, 1.6, -0.6],
                        [0.5, -1.9, -0.9]
                         ])[:,(0,1,2)]   # passing


seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((3,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)

start = seed_points[1,:]
goal = seed_points[2,:]

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), np.zeros((3,)) ))
    
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])
context = diagram.CreateDefaultContext()
q_star = np.zeros(3)
def check_empty_polytope(C, d):
    prog = MathematicalProgram()
    t = prog.NewContinuousVariables(C.shape[1], "t")

    prog.AddLinearConstraint(C,-np.inf*np.ones(C.shape[0]), d, t)
    prog.AddBoundingBoxConstraint(limits_t[0], limits_t[1], t)
    result = MosekSolver().Solve(prog)
    return result.is_success(), result.get_solution_result()

In [ ]:
# RUN THIS CELL IF YOU WANT TO DO SNOPT IRIS

# regions = []

# iris_options = IrisOptionsRationalSpace()
# iris_options.require_sample_point_is_contained = True
# iris_options.iteration_limit = 20
# iris_options.configuration_space_margin = 1e-5
# iris_options.max_faces_per_collision_pair = 60
# iris_options.termination_threshold = -1
# iris_options.q_star = np.zeros(3)
# iris_options.relative_termination_threshold = 0.05
# iris_options.enable_ibex = False
# #deprecated
# iris_options.certify_region_with_sos_during_generation = False
# iris_options.certify_region_with_sos_after_generation = False

# for i, s in enumerate(seed_points):
#     plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
#     if False:
#         #starting_hpolyhedron = regions[i-1]
#         r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
#                                               iris_options, starting_hpolyhedron)
#     else:
#         r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
#     regions.append(r)
#     print(f'Completed region: {i+1}/{len(seed_points)}')
#     print(f"Sample point contained = {np.all(r.A()@s <= r.b())}")
#     print(f"Sample point contained componentwise = {r.A()@s <= r.b()}")
#     print()

In [ ]:
# if do_viz:
#     visualizer.plot_regions(regions, ellipses=None, region_suffix='original')

## Create default region

In [ ]:
regions_default = []
max_vects = []
min_vects = []
vector_bisection_options_list = []
bilinear_alteration_options_list = []
interleaved_options_list = []

#parameters for creating a default region. Currently set to create a box
num_perm_dim = 3
num_rot = 1

for i, s in enumerate(seed_points):
    regions_default.append(GenerateSeedingPolytope(s, num_perm_dim, num_rot))
    
    eps_min_vect = FindEpsilonLowerVector(regions_default[i].A(), 
                                      regions_default[i].b(), 
                                      limits_t[0], limits_t[1], s)
    
    min_vects.append(eps_min_vect)
    
    eps_max_vect = FindMaxEpsForAllIneqs(plant, plant.GetMyMutableContextFromRoot(context), 
                                q_star, regions_default[i].A(), regions_default[i].b(), 
                                         eps_min_vect, limits_t[0], limits_t[1], s)
    max_vects.append(eps_max_vect)
    
    vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
    vector_bisection_search_options.max_iters = 10
    vector_bisection_search_options.max_feasible_iters = 5
    vector_bisection_search_options.search_d = True
    vector_bisection_search_options.epsilon_min = eps_min_vect
    vector_bisection_search_options.epsilon_max = eps_max_vect
    vector_bisection_search_options.num_threads = -1
    vector_bisection_options_list.append(vector_bisection_search_options)
    
    bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()
    bilinear_alternation_options.max_iters = 20
    bilinear_alternation_options.lagrangian_backoff_scale = 0
    bilinear_alternation_options.polytope_backoff_scale = 0
    bilinear_alternation_options.num_threads = -1
    bilinear_alteration_options_list.append(bilinear_alternation_options)
    
    interleaved_options = rational_forward_kinematics.InterleavedRegionSearchOptions()
    interleaved_options.vector_bisection_search_options = vector_bisection_search_options
    interleaved_options.bilinear_alternation_options = bilinear_alternation_options
    #actually does round robin
    interleaved_options.use_vector_bisection_search = True
    interleaved_options.max_method_switch = 1
    interleaved_options_list.append(interleaved_options)
    
    
num_rounds = 1

if do_viz:
    visualizer.plot_regions(regions_default, ellipses=None, region_suffix='_default')
    

In [12]:
i = 0
s = seed_points[i,:]
C, d = regions_default[i].A(), regions_default[i].b()
vector_bisection_search_options = vector_bisection_options_list[i]
interleaved_options = interleaved_options_list[i]

In [13]:
print(vector_bisection_search_options.epsilon_max)
print(vector_bisection_search_options.epsilon_min)
print(vector_bisection_search_options.epsilon_max-
     vector_bisection_search_options.epsilon_min)

[-4.16333634e-17 -2.77555756e-17  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -2.77555756e-17]
[-0.10000001 -0.10000001 -0.10000001 -0.09999999 -0.09999999 -0.09999999]
[0.10000001 0.10000001 0.10000001 0.09999999 0.09999999 0.09999999]


In [14]:
#plot the new polytopes which each individual face moved
if do_viz:
    for i, r in enumerate(regions_default):
        C, d = r.A(), r.b()
        eps_max = vector_bisection_options_list[i].epsilon_max
        r_moved_list = []
        for j in range(len(d)):
            d_new = d.copy()
            d_new[j] += eps_max[j]
            r_moved_list.append(HPolyhedron(C, d_new))
        if do_viz:
            visualizer.plot_regions(r_moved_list, ellipses=None, region_suffix=f'_default_{i}_maxed')
        if do_viz:
            visualizer.plot_regions([HPolyhedron(C, d+0.99*eps_max)], ellipses=None, region_suffix=f'_all_{i}_maxed')

## Run Cspace Free Region

In [15]:
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)
filtered_collision_pairs = set()
solver_options = mp.SolverOptions()
# solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

In [16]:
cspace_free_region_solution_interleaved = cspace_free_region.InterleavedCSpacePolytopeSearch(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, num_rounds, 
                                                                     interleaved_options,
                                                                     solver_options, s)

[2022-02-08 20:57:54.472] [console] [info] Starting bisections
[2022-02-08 20:58:08.902] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 20:58:12.560] [console] [info] bilinear alt on d succeeded
[2022-02-08 20:58:12.623] [console] [info] Ineq 1/6 in round 1
[2022-02-08 20:58:12.651] [console] [info] output of eps_redundant_max is 1.5899722811905947
[2022-02-08 20:58:12.652] [console] [info] current gap is epsilon_gap=0.06858450234557706
[2022-02-08 20:58:18.667] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 20:58:22.055] [console] [info] bilinear alt on d succeeded
[2022-02-08 20:58:22.158] [console] [info] epsilon=-0.03429225117278853 is feasible
[2022-02-08 20:58:22.187] [console] [info] output of eps_redundant_max is 1.527420497329439
[2022-02-08 20:58:22.188] [console] [info] reset eps_min=-0.006032718484421162, eps_max=0.0
[2022-02-08 20:58:22.189] [console] [info] current gap is epsilon_gap=0.006032718484421162
[202

[2022-02-08 21:00:08.335] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 21:00:08.375] [console] [info] output of eps_redundant_max is 0.7836259619264941
[2022-02-08 21:00:08.376] [console] [info] reset eps_min=-0.000635169919091072, eps_max=-4.163336342344337e-17
[2022-02-08 21:00:08.376] [console] [info] current gap is epsilon_gap=0.0006351699190910304
[2022-02-08 21:00:14.693] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 21:00:18.989] [console] [info] bilinear alt on d succeeded
[2022-02-08 21:00:19.074] [console] [info] epsilon=-0.0003175849595455568 is feasible
[2022-02-08 21:00:19.075] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 21:00:19.112] [console] [info] output of eps_redundant_max is 0.7833083473815534
[2022-02-08 21:00:19.113] [console] [info] reset eps_min=-0.00031755537415040225, eps_max=-4.163336342344

[2022-02-08 21:02:02.479] [console] [info] reset eps_min=-2.025001514896485e-05, eps_max=0.0
[2022-02-08 21:02:02.480] [console] [info] Ineq 5/6 in round 1
[2022-02-08 21:02:02.528] [console] [info] output of eps_redundant_max is -8.259771603384296
[2022-02-08 21:02:02.530] [console] [warning] round robin bisection search failed: epsilon max is less than epsilon min.
 difference is = 
  0.0121122
 0.0975384
  0.394508
-2.025e-05
 0.0590251
-0.0998424
[2022-02-08 21:02:02.531] [console] [info] Ineq 6/6 in round 1
[2022-02-08 21:02:02.562] [console] [info] output of eps_redundant_max is 0.6551200129874322
[2022-02-08 21:02:02.563] [console] [info] current gap is epsilon_gap=0.0998423653755712
[2022-02-08 21:02:07.858] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_5_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_4_collision)
(iiwa7_twoDOF::

[2022-02-08 21:03:16.898] [console] [info] Lagrangian step time 5.09 s


In [18]:
r = HPolyhedron(np.vstack([starting_poly.A(),cspace_free_region_solution_interleaved.C]),
                np.hstack([starting_poly.b(), cspace_free_region_solution_interleaved.d])
                )
e = Hyperellipsoid(np.linalg.inv(cspace_free_region_solution_interleaved.P), 
                   cspace_free_region_solution_interleaved.q)
if do_viz:
    visualizer.plot_regions([r], 
                            ellipses=[e], region_suffix='_interleaved_single')

In [23]:
r = HPolyhedron(np.vstack([starting_poly.A(),cspace_free_region_solution_round_robin.C]),
                np.hstack([starting_poly.b(), cspace_free_region_solution_round_robin.d])
                )
e = Hyperellipsoid(np.linalg.inv(cspace_free_region_solution_round_robin.P), cspace_free_region_solution_round_robin.q)
if do_viz:
    visualizer.plot_regions([r], 
                            ellipses=[e], region_suffix='_round_robin_single')

In [19]:
end_ind = 2
seed_point_list = [s for s in seed_points[:end_ind]] 
C_mats = [r.A() for r in regions_default[:end_ind]]
d_vects = [r.b() for r in regions_default[:end_ind]]
cspace_free_region_solution_interleaved_vect = cspace_free_region.InterleavedCSpacePolytopeSearchForSeedPoints(
                                                                    q_star,
                                                                     filtered_collision_pairs,
                                                                     C_mats, d_vects, num_rounds, 
                                                                     interleaved_options_list[:end_ind],
                                                                     solver_options, seed_point_list)
# cspace_free_region_solution_round_robin_vect = cspace_free_region.CspacePolytopeRoundRobinBisectionSearchForSeedPoints(
#                                                                     q_star,
#                                                                      filtered_collision_pairs,
#                                                                      C_mats, d_vects, num_rounds, 
#                                                                      vector_bisection_options_list[:end_ind],
#                                                                      solver_options, seed_point_list)

[2022-02-08 21:08:03.748] [console] [info] Starting bisections
[2022-02-08 21:08:03.751] [console] [info] Starting bisections
[2022-02-08 21:08:18.753] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 21:08:18.756] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 21:08:22.125] [console] [info] bilinear alt on d succeeded
[2022-02-08 21:08:22.161] [console] [info] Ineq 1/6 in round 1
[2022-02-08 21:08:22.162] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 21:08:22.164] [console] [info] output of eps_redundant_max is 2.317612112878893
[2022-02-08 21:08:22.165] [console] [info] current gap is epsilon_gap=1.282997606962152
[2022-02-08 21:08:23.473] [console] [info] bilinear alt on d succeeded
[2022-02-08 21:08:23.649] [console] [info] Ineq 1/6 in round 1
[2022-02-08 21:08:23.963] [console] [info] output of eps_redundant_max is 1.126644757737606
[2022-02-08 21:08:23

[2022-02-08 21:09:18.020] [console] [info] bilinear alt on d succeeded
[2022-02-08 21:09:18.077] [console] [info] epsilon=0.13066225294407283 is feasible
[2022-02-08 21:09:18.091] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 21:09:18.114] [console] [info] output of eps_redundant_max is 2.0869498702575386
[2022-02-08 21:09:18.115] [console] [info] reset eps_min=0.1306622535310401, eps_max=1.182997617871838
[2022-02-08 21:09:18.116] [console] [info] current gap is epsilon_gap=1.0523353643407978
[2022-02-08 21:09:21.849] [console] [info] bilinear alt on d succeeded
[2022-02-08 21:09:21.993] [console] [info] epsilon=-0.006920120756180405 is feasible
[2022-02-08 21:09:22.000] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 21:09:22.002] [console] [info] output of eps_redundant_max is 1.4478856454791817
[2022-02-08 21:09:22.004] [console] [info] reset eps_min

[2022-02-08 21:10:09.690] [console] [info] epsilon=0.6807962924187333 is infeasible
[2022-02-08 21:10:09.691] [console] [info] current gap is epsilon_gap=0.16479014505759026
[2022-02-08 21:10:11.700] [console] [info] bilinear alt on d succeeded
[2022-02-08 21:10:11.801] [console] [info] epsilon=-0.004809351841768193 is feasible
[2022-02-08 21:10:11.809] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 21:10:11.811] [console] [info] output of eps_redundant_max is 1.0431420640465343
[2022-02-08 21:10:11.819] [console] [info] reset eps_min=-0.004809350818094915, eps_max=-4.163336342344337e-17
[2022-02-08 21:10:11.820] [console] [info] current gap is epsilon_gap=0.0048093508180948735
[2022-02-08 21:10:19.259] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box4_Collision@00000000046864b8, iiwa7_oneDOF::iiwa_oneDOF_link_5_collision)
(box_scene::box4_Collision@00000000046864b8, iiwa7

[2022-02-08 21:11:05.858] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_5_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_6_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-08 21:11:05.859] [console] [info] epsilon=-0.03465082011526713 is infeasible
[2022-02-08 21:11:05.860] [console] [info] current gap is epsilon_gap=0.0653491484076327
[2022-02-08 21:11:08.045] [console] [info] bilinear alt on d succeeded
[2022-02-08 21:11:08.214] [console] [info] epsilon=-0.0003005844708324698 is feasible
[2022-02-08 21:11:08.223] [console] [info] epsilon exceeds components of epsilon max in some components, increasing

[2022-02-08 21:12:07.674] [console] [info] output of eps_redundant_max is 1.2927084564258218
[2022-02-08 21:12:07.674] [console] [info] reset eps_min=0.1684174755407793, eps_max=0.4227932187381618
[2022-02-08 21:12:07.675] [console] [info] current gap is epsilon_gap=0.2543757431973825
[2022-02-08 21:12:11.646] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@0000000004676818, iiwa7_twoDOF::iiwa_twoDOF_link_6_collision)
(box_scene::box2_Collision@0000000004676818, iiwa7_twoDOF::iiwa_twoDOF_link_7_collision)

[2022-02-08 21:12:11.647] [console] [info] epsilon=0.2956053471394705 is infeasible
[2022-02-08 21:12:11.648] [console] [info] current gap is epsilon_gap=0.12718787159869122
[2022-02-08 21:12:12.465] [console] [info] bilinear alt on d succeeded
[2022-02-08 21:12:12.538] [console] [info] epsilon=-0.012499965934919421 is feasible
[2022-02-08 21:12:12.541] [console] [info] epsilon exceeds components of epsilon max in some compo

[2022-02-08 21:13:16.963] [console] [info] Starting Alternations
[2022-02-08 21:13:17.134] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@0000000004676818, iiwa7_twoDOF::iiwa_twoDOF_link_7_collision)

[2022-02-08 21:13:17.136] [console] [info] epsilon=0.2662925193099832 is infeasible
[2022-02-08 21:13:17.136] [console] [info] Ineq 5/6 in round 1
[2022-02-08 21:13:17.164] [console] [info] output of eps_redundant_max is 0.7155394343578492
[2022-02-08 21:13:17.165] [console] [info] current gap is epsilon_gap=0.09999993986757166
[2022-02-08 21:13:21.915] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_5_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_5_collision, iiwa7_oneDOF::iiwa_oneDOF_link_5_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_5_

[2022-02-08 21:14:24.650] [console] [info] max(power(det(P), 1/4))=0.3180190804181828, solver_time 0.02165508270263672
[2022-02-08 21:14:24.656] [console] [info] cost improvement 0.00856797858307523
[2022-02-08 21:14:28.134] [console] [info] Iter: 4, polytope step cost -0.01874465569686926, solver time 1.0309679508209229
[2022-02-08 21:14:32.958] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 21:14:32.960] [console] [info] Lagrangian step time 4.763 s
[2022-02-08 21:14:33.008] [console] [info] max(power(det(P), 1/4))=0.32631123467642625, solver_time 0.03805208206176758
[2022-02-08 21:14:33.010] [console] [info] cost improvement 0.008292154258243478
[2022-02-08 21:14:37.736] [console] [info] Iter: 5, polytope step cost -0.018361053699305033, solver time 2.4040298461914062
[2022-02-08 21:14:42.563] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 21:14:42.566] [console] [info] Lagrangian step time 4.759 s
[2022-02-08 21:14:42.

In [20]:
regions_round_robin = []
ellipses_round_robin = []
for sol in cspace_free_region_solution_interleaved_vect:
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C]),
                np.hstack([starting_poly.b(), sol.d])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)
if do_viz:
    visualizer.plot_regions(regions_round_robin, 
                            ellipses=ellipses_round_robin, region_suffix='_interleaved')

# Don't run after here

In [22]:
cspace_free_region_solution_round_robin = cspace_free_region.CspacePolytopeRoundRobinBisectionSearch(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, num_rounds, 
                                                                     vector_bisection_search_options,
                                                                     solver_options, s)

[2022-02-08 19:29:47.629] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:29:49.330] [console] [info] bilinear alt on d succeeded
[2022-02-08 19:29:49.358] [console] [info] Ineq 1/6 in round 1
[2022-02-08 19:29:49.359] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 19:29:49.379] [console] [info] output of eps_redundant_max is 1.8224695040938002
[2022-02-08 19:29:49.381] [console] [info] current gap is epsilon_gap=0.8341784992408996
[2022-02-08 19:29:52.187] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:29:53.997] [console] [info] bilinear alt on d succeeded
[2022-02-08 19:29:54.033] [console] [info] epsilon=0.3170892670967813 is feasible
[2022-02-08 19:29:54.034] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 19:29:54.051] [console] [info] output of eps_redundant_max is 1.3733133612699

[2022-02-08 19:30:42.509] [console] [info] epsilon=-0.024665965387383592 is infeasible
[2022-02-08 19:30:42.510] [console] [info] current gap is epsilon_gap=0.07470767376292238
[2022-02-08 19:30:45.547] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(box_scene::box2_Collision@00005642fcbecef8, iiwa7_twoDOF::iiwa_twoDOF_link_7_collision)

[2022-02-08 19:30:45.549] [console] [info] epsilon=-0.062019802268844786 is infeasible
[2022-02-08 19:30:45.549] [console] [info] current gap is epsilon_gap=0.03735383688146119
[2022-02-08 19:30:48.945] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:30:50.770] [console] [info] bilinear alt on d succeeded
[2022-02-08 19:30:50.810] [console] [info] epsilon=-0.08069672070957538 is feasible
[2022-02-08 19:30:50.811] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 19:30:50.821] [console] [info] output of eps_redundant_max is

[2022-02-08 19:31:36.939] [console] [info] current gap is epsilon_gap=0.01641079987684163
[2022-02-08 19:31:40.455] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_3_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-08 19:31:40.457] [console] [info] epsilon=-0.07303929070077358 is infeasible
[2022-02-08 19:31:40.458] [console] [info] current gap is epsilon_gap=0.008205399938420815
[2022-02-08 19:31:43.692] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:31:45.558] [console] [info] bilinear alt on d succeeded
[2022-02-08 19:31:45.600] [console] [info] epsilon=-0.07714199066998399 is feasible
[2022-02-08 19:31:45.601] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-08 19:31:45.627] [console] [info] output of eps_redundant_max 

In [29]:
cspace_free_region_solution_bilinear = cspace_free_region.CspacePolytopeBilinearAlternation(q_star,
                                                                     filtered_collision_pairs,
                                                                     cspace_free_region_solution_round_robin.C,
                                                                    0.75*cspace_free_region_solution_round_robin.d,
                                                                     bilinear_alternation_options,
                                                                     solver_options, s)

[2022-02-08 19:34:36.721] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:34:36.723] [console] [info] Lagrangian step time 3.045 s
[2022-02-08 19:34:36.751] [console] [info] max(power(det(P), 1/4))=0.2071381570004865, solver_time 0.018611907958984375
[2022-02-08 19:34:36.753] [console] [info] cost improvement inf
[2022-02-08 19:34:38.615] [console] [info] Iter: 0, polytope step cost -0.008828768121858371, solver time 0.48531293869018555
[2022-02-08 19:34:41.859] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:34:41.860] [console] [info] Lagrangian step time 3.218 s
[2022-02-08 19:34:41.881] [console] [info] max(power(det(P), 1/4))=0.2105033555416147, solver_time 0.013540029525756836
[2022-02-08 19:34:41.882] [console] [info] cost improvement 0.0033651985411282115
[2022-02-08 19:34:43.780] [console] [info] Iter: 1, polytope step cost -0.008522369595631428, solver time 0.532606840133667
[2022-02-08 19:34:46.920] [console

[2022-02-08 19:36:00.640] [console] [info] Iter: 16, polytope step cost -0.00557567747292513, solver time 0.38878607749938965
[2022-02-08 19:36:04.137] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:36:04.141] [console] [info] Lagrangian step time 3.472 s
[2022-02-08 19:36:04.172] [console] [info] max(power(det(P), 1/4))=0.24350116763464993, solver_time 0.01593494415283203
[2022-02-08 19:36:04.174] [console] [info] cost improvement 0.001400599503426303
[2022-02-08 19:36:05.966] [console] [info] Iter: 17, polytope step cost -0.005466929346269455, solver time 0.3860509395599365
[2022-02-08 19:36:09.113] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:36:09.114] [console] [info] Lagrangian step time 3.119 s
[2022-02-08 19:36:09.142] [console] [info] max(power(det(P), 1/4))=0.24485582644352566, solver_time 0.00913095474243164
[2022-02-08 19:36:09.145] [console] [info] cost improvement 0.001354658808875725
[2022-02-08 19:3

In [31]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution_bilinear.C,
                                         cspace_free_region_solution_bilinear.d)], 
                            ellipses=None, region_suffix='_bilinear')

In [15]:
regions_round_robin = []
ellipses_round_robin = []
for sol in cspace_free_region_solution_round_robin_vect:
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C]),
                np.hstack([starting_poly.b(), sol.d])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)
if do_viz:
    visualizer.plot_regions(regions_round_robin, 
                            ellipses=ellipses_round_robin, region_suffix='_round_robin')

In [22]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution.C, 0.5*cspace_free_region_solution.d)], 
                            ellipses=None, region_suffix='_round_robin')

In [26]:
cspace_free_region_solution_vector_bisection = cspace_free_region.CspacePolytopeBisectionSearchVector(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, 
                                                                     vector_bisection_search_options,
                                                                     solver_options, s)

[2022-02-07 15:27:37.125] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-07 15:27:37.194] [console] [info] max(power(det(P), 1/4))=-nan, solver_time 0.05913114547729492
[2022-02-07 15:27:40.385] [console] [info] bilinear alt on d succeeded
[2022-02-07 15:27:40.411] [console] [info] max(power(det(P), 1/4))=1.195363453968912e-06, solver_time 0.014796972274780273
[2022-02-07 15:27:45.796] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_3_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-07 15:27:45.797] [console] [info] total iter=1, feasible iter = 0
[2022-02-07 15:27:51.001] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_

In [29]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution_vector_bisection.C, 
                                         cspace_free_region_solution_vector_bisection.d)], 
                            ellipses=None, region_suffix='_vector_bisection')

(True, <SolutionResult.kSolutionFound: 0>)


In [96]:
i

2

In [84]:
vector_bisection_search_options_list = []
scalar_bisection_search_options_list = []
bilinear_alternation_options_list = []
integrated_search_options_list = []


search_tuples = []

for i, r in enumerate(regions_default):
    vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
    vector_bisection_search_options.max_iters = 20
    vector_bisection_search_options.max_feasible_iters = 5
    vector_bisection_search_options.search_d = True
    vector_bisection_search_options.epsilon_min = FindEpsilonLowerVector(r.A(), r.b(),
                                                             limits_t[0], limits_t[1], seed_points[i,:])
    vector_bisection_search_options.epsilon_max = 0.25*FindEpsilonUpperVector(r.A(), 
                                      r.b(), 
                                      limits_t[0], limits_t[1])
    vector_bisection_search_options_list.append(vector_bisection_search_options)
    
    scalar_bisection_search_options = rational_forward_kinematics.BinarySearchOption()
    scalar_bisection_search_options.epsilon_max = 1
    scalar_bisection_search_options.max_iters = 15
    scalar_bisection_search_options.search_d = True
    scalar_bisection_search_options.epsilon_min = FindEpsilonLower(r.A(), r.b(),
                                                         limits_t[0], limits_t[1],
                                                         seed_points[i,:])
    scalar_bisection_search_options_list.append(scalar_bisection_search_options)
    
    
    bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()
    bilinear_alternation_options.max_iters = 20
    bilinear_alternation_options.lagrangian_backoff_scale = 1e-6
    bilinear_alternation_options.polytope_backoff_scale = 1e-6
    bilinear_alternation_options_list.append(bilinear_alternation_options)
    
    integrated_search_options = rational_forward_kinematics.InterleavedRegionSearchOptions()
    integrated_search_options.vector_bisection_search_options = vector_bisection_search_options
    integrated_search_options.scalar_binary_search_options = scalar_bisection_search_options
    integrated_search_options.bilinear_alternation_options = bilinear_alternation_options
    integrated_search_options.max_method_switch = 2
    integrated_search_options.use_vector_bisection_search = True
    integrated_search_options_list.append(integrated_search_options)
    
    search_tuples.append((r, integrated_search_options, seed_points[i,:]))

AttributeError: module 'pydrake.multibody.rational_forward_kinematics' has no attribute 'IntegratedRegionSearchOptions'

In [26]:
def launch_integrated_search(region, integrated_search_options, seed_point):
    return cspace_free_region.IntegratedCSpacePolytopeSearch(q_star,filtered_collision_pairs,
                                                      region.A(), region.b(),
                                                     integrated_search_options, 
                                                      solver_options, seed_point
                                                     )
def launch_vector_bisection_search(region, integrated_search_options, seed_point):
    return cspace_free_region.IntegratedCSpacePolytopeSearch(q_star,filtered_collision_pairs,
                                                      region.A(), region.b(),
                                                     integrated_search_options, 
                                                      solver_options, seed_point
                                                     )

In [16]:
ret_vals = []
for tup in search_tuples:
    ret_vals.append(launch_integrated_search(*tup))

NameError: name 'search_tuples' is not defined

In [17]:
i = 1
r = regions[i]
s = seed_points_q[i]
vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
vector_bisection_search_options.max_iters = 1
vector_bisection_search_options.max_feasible_iters = 20
vector_bisection_search_options.search_d = True
vector_bisection_search_options.epsilon_min = FindEpsilonLowerVector(r.A(), r.b(),
                                                         limits_t[0], limits_t[1], seed_points[i,:])
vector_bisection_search_options.epsilon_max = 0.98*vector_bisection_search_options.epsilon_min #np.zeros(r.b().shape[0])

In [18]:
d_final, _ = cspace_free_region.CspacePolytopeBisectionSearchVector(q_star,set(),
                                                      r.A(), r.b(),
                                                     vector_bisection_search_options, 
                                                      solver_options, seed_points_q)

[2022-02-05 15:10:26.860] [console] [info] bilinear alt on d succeeded
[2022-02-05 15:10:26.892] [console] [info] Solver time 2.400635004043579


In [20]:
r_new = HPolyhedron(r.A(), d_final)
if do_viz:
    visualizer.plot_regions([r_new], ellipses=None, region_suffix='_new')

In [17]:
planes = cspace_free_region.separating_planes()

<Expression "(b_constant0 + (t[0] * b_coeff0(0)) + (t[1] * b_coeff0(1)) + (t[2] * b_coeff0(2)))">

In [16]:
#refine with CPP code:
cpp_editted_scalar_regions = []
for i in range(seed_points.shape[0]):
    s = seed_points[i,:]
    region_init = regions[i]
    region_init, region = iris_scalar_handle(s, region_init)
    cpp_editted_scalar_regions.append(region)
    print(f"Completed {i+1}/{seed_points.shape[0]}")

[2022-01-31 15:40:42.251] [console] [info] Solver time 2.6619720458984375
[2022-01-31 15:40:44.826] [console] [info] Solver time 1.0261502265930176


[IRIS +  CERT]: Eps min -0.2652286679481278
[IRIS +  CERT]: Region found, vol =  0.6819787390374958 , halfaxis:  [4.86595118 2.17555524 0.58020292]


[2022-01-31 15:40:48.683] [console] [info] Solver time 1.619724988937378
[2022-01-31 15:40:48.733] [console] [info] epsilon=-0.1326143339740639 is infeasible
[2022-01-31 15:40:53.321] [console] [info] Solver time 1.117055892944336
[2022-01-31 15:40:53.401] [console] [info] epsilon=-0.19892150096109587 is feasible
[2022-01-31 15:40:53.402] [console] [info] reset eps_min=0.0, eps_max=0.06630716698703196


Completed 1/5
[IRIS +  CERT]: Eps min -0.29420562277149487
[IRIS +  CERT]: Region found, vol =  0.5535987103063078 , halfaxis:  [4.69487125 2.58579857 0.62326848]


[2022-01-31 15:41:02.811] [console] [info] Solver time 3.412208080291748
[2022-01-31 15:41:05.195] [console] [info] Solver time 0.7390580177307129
[2022-01-31 15:41:10.948] [console] [info] Solver time 1.3550069332122803
[2022-01-31 15:41:11.041] [console] [info] epsilon=-0.14710281138574743 is feasible
[2022-01-31 15:41:11.042] [console] [info] reset eps_min=0.0, eps_max=0.14710281138574743
[2022-01-31 15:41:16.878] [console] [info] Solver time 2.747184991836548
[2022-01-31 15:41:16.951] [console] [info] epsilon=0.07355140569287372 is infeasible
[2022-01-31 15:41:24.801] [console] [info] Solver time 3.2756950855255127


...

Completed 4/5


[2022-01-31 15:42:31.487] [console] [info] Solver time 4.768555164337158


[IRIS +  CERT]: Eps min -0.33868521538628993
[IRIS +  CERT]: Region found, vol =  0.43617411313252785 , halfaxis:  [3.98248604 1.65632853 1.45588804]


[2022-01-31 15:42:34.253] [console] [info] Solver time 0.7434499263763428
[2022-01-31 15:42:42.592] [console] [info] Solver time 1.8796329498291016
[2022-01-31 15:42:42.727] [console] [info] epsilon=-0.16934260769314496 is feasible
[2022-01-31 15:42:42.728] [console] [info] reset eps_min=0.0, eps_max=0.16934260769314502
[2022-01-31 15:42:52.439] [console] [info] Solver time 2.5359718799591064
[2022-01-31 15:42:52.575] [console] [info] epsilon=0.08467130384657251 is feasible
[2022-01-31 15:42:52.576] [console] [info] reset eps_min=0.0, eps_max=0.08467130384657251


Completed 5/5


In [40]:
if do_viz:
    visualizer.plot_regions(cpp_editted_regions, ellipses=None, region_suffix='cpp_editted_regions')

In [44]:
#setup refine with python
iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-5,
    'iris_plane_pullback': 1e-4,
    'iris_max_faces': -1
}
iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, **iris_kwargs)
regions, ellipses = iris_generator.iris_in_rational_space(seed_points)
iris_generator.initalize_certifier(plane_order = 1, strict_pos_tol = 1e-4)

snopt_example=[ 0.35244668 -0.19250022 -0.17979872], growth = 1936025801.4851828
snopt_example=[ 0.23144445 -0.04062289 -0.46224621], growth = 2688883094.3830557
snopt_example=[ 0.2001245  -0.10137361 -0.26895992], growth = 1226658660.3889592
snopt_example=[0.         0.         0.22974536], growth = 527829310.38532484
snopt_example=[-0.18114806  0.0938193   0.        ], growth = 416166830.35149866
snopt_example=[-0.1760171   0.09533373  0.        ], growth = 400705370.23392135
snopt_example=[ 0.2157998  -0.11140991 -0.25108731], growth = 1220265554.750688
0
snopt_example=[ 0.16305709  0.71849721 -0.56308786], growth = 2.899642778747243
snopt_example=[-0.60852286 -1.18066191 -0.12769203], growth = 0.6628366692873271
1
Time:   1.61 	Volume:   0.56 	Center: [ 0.18683057  0.06844461 -0.0630242 ]
snopt_example=[ 0.81716573  0.47417491 -0.38295522], growth = 2389296154.6526375
snopt_example=[ 0.77694698  0.54947603 -0.40266231], growth = 1702365874.8724155
snopt_example=[ 0.44441592  0.7602

{<pydrake.geometry.optimization.HPolyhedron at 0x7f9f7db27530>: <certified_iris_generator.RegionCertificationProblem at 0x7f9f7f87f3d0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f9f7f379e70>: <certified_iris_generator.RegionCertificationProblem at 0x7f9f7f87f730>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f9f7da92e70>: <certified_iris_generator.RegionCertificationProblem at 0x7f9f7f87fdf0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f9f7f6fc530>: <certified_iris_generator.RegionCertificationProblem at 0x7f9f7f87fd90>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7f9f7dc17570>: <certified_iris_generator.RegionCertificationProblem at 0x7f9f7f87c550>}

In [45]:
if do_viz:
    visualizer.plot_regions(iris_generator.regions, region_suffix = '_python_snopt')

In [ ]:
# refine with python
do_linesearch_cert = True
if do_linesearch_cert:
    iris_generator.certify_and_adjust_regions_by_linesearch(1e-5)

Starting Region 1/5
min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.56916636 0.77870386 0.56916636 0.56916636 0.77870386 0.56916636
 0.22000147 0.25927221 0.17511844 0.11487268 0.10200084 0.10008813
 0.1746615  0.31397342 0.33108215]
max_eps = [1.13833271 1.55740772 1.13833271 1.13833271 1.55740772 1.13833271
 0.44000293 0.51854441 0.35023687 0.22974536 0.20400167 0.20017626
 0.349323   0.62794684 0.66216431]
Biggest difference: 1.5574077246549023
Smallest difference: 0.20017626488520598
Fixed eps is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.28458318 0.38935193 0.28458318 0.28458318 0.38935193 0.28458318
 0.11000073 0.1296361  0.08755922 0.05743634 0.05100042 0.05004407
 0.08733075 0.15698671 0.16554108]
max_eps = [0.56916636 0.77870386 0.56916636 0.56916636 0.77870386 0.56916636
 0.22000147 0.25927221 0.17511844 0.11487268 0.10200084 0.10008813
 0.1746615  0.31397342 0.33108215]
Biggest difference: 0.7787038623274511
Smallest 

In [ ]:
if do_viz:
    visualizer.plot_regions(iris_generator.linesearch_regions, region_suffix = '_new_by_linesearch')